# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
df = pd.read_csv("Output/Weather Data.csv")

df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,São Filipe,14.8961,-24.4956,79.63,68,13,8.12,CV,-3600
1,Kosa,59.9445,54.9977,34.29,94,63,5.35,RU,18000
2,Atuona,-9.8000,-139.0333,78.35,77,13,17.65,PF,-34200
3,Vilcún,-38.6500,-72.2333,48.94,87,93,1.72,CL,-10800
4,Rikitea,-23.1203,-134.9692,75.16,79,90,18.61,PF,-32400
...,...,...,...,...,...,...,...,...,...
554,Hammerfest,70.6634,23.6821,30.65,100,75,5.75,NO,7200
555,Siocon,7.7061,122.1353,79.18,85,100,2.73,PH,28800
556,Marzuq,14.4000,46.4667,68.02,34,20,2.13,YE,10800
557,Puerto Baquerizo Moreno,-0.9000,-89.6000,69.01,86,95,10.98,EC,-21600


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [8]:
# Store latitude and longitude in locations
coordinates=[]

for index, row in df.iterrows():
    coordinates.append((row['Latitude'],row['Longitude']))
    
coordinates

[(14.8961, -24.4956),
 (59.9445, 54.9977),
 (-9.8, -139.0333),
 (-38.65, -72.2333),
 (-23.1203, -134.9692),
 (-46.1927, 168.8643),
 (60.7333, 77.5889),
 (-33.9258, 18.4232),
 (61.9353, 5.1136),
 (47.6666, -52.7314),
 (29.15, 78.2667),
 (50.6789, 156.125),
 (-33.5906, 26.891),
 (19.7297, -155.09),
 (-53.15, -70.9167),
 (-20.5167, 57.5167),
 (42.5357, -113.7928),
 (63.7506, -68.5145),
 (55.7992, -4.3929),
 (41.3712, -73.414),
 (69.4541, -133.0374),
 (77.484, -69.3632),
 (-33.65, 115.3333),
 (48.8264, 58.1442),
 (11.2842, 49.1816),
 (64.2539, -15.2082),
 (20.5022, -10.0711),
 (64.3833, -173.3),
 (-3.9014, -38.3911),
 (25.5198, 90.2201),
 (-46.6, 168.3333),
 (22.8909, -109.9124),
 (63.05, 179.3167),
 (-54.8, -68.3),
 (53.7453, 62.0708),
 (68.0546, 166.4372),
 (5.4112, 100.3354),
 (12.6, 37.4667),
 (6.9833, 35.5833),
 (42.6001, -73.9662),
 (71.2906, -156.7887),
 (-34.4187, 19.2345),
 (28.5004, -13.8627),
 (-28.7667, 114.6),
 (32.6667, -17.1),
 (56.9156, 53.3221),
 (62.0989, 126.6992),
 (-0.

In [9]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

In [10]:
# Assign the marker layer to a variable
markers = gmaps.heatmap_layer(coordinates, weights=df['Humidity'],point_radius=15)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
tempmin = 70
tempmax = 80

df_temp_filt = df.loc[(df["Max Temperature"]<tempmax)&(df["Max Temperature"]>tempmin)]
df_temp_filt

windspeedmax = 10

df_windspeed_filt = df_temp_filt.loc[df_temp_filt['Wind Speed']<windspeedmax]
df_windspeed_filt

df_final = df_windspeed_filt.loc[df_windspeed_filt['Cloudiness'] == 0]
df_final

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
248,Dezful,32.3811,48.4058,74.39,20,0,4.88,IR,12600
293,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,14400
326,Atar,20.5169,-13.0499,79.54,17,0,7.09,MR,0
382,Camargo,27.6667,-105.1667,79.95,18,0,3.53,MX,-21600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
import requests
import json
import pprint

hotel_df = df_final
hotel_df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
248,Dezful,32.3811,48.4058,74.39,20,0,4.88,IR,12600
293,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,14400
326,Atar,20.5169,-13.0499,79.54,17,0,7.09,MR,0
382,Camargo,27.6667,-105.1667,79.95,18,0,3.53,MX,-21600


In [14]:
hotel_list = []
hlat=[]
hlng=[]

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
        hotelname = response["results"][0]["name"]
        CityName = row['City Name']
        print(hotelname)
        hotel_list.append(hotelname)
        hlat.append(response["results"][0]["geometry"]["location"]["lat"])
        hlng.append(response["results"][0]["geometry"]["location"]["lng"])

    except IndexError:
        print(f"No Hotel within 5km of this city")
        hotel_list.append("No Hotel within 5km of this city")
        hlat.append("")
        hlng.append("")

Dezful Tourism Hotel
Salalah Gardens Hotel
Auberge et Camping Inimi
Hotel Santa Fe


In [15]:
# Store Dataframe under a new name and reset index
hotel_reset = hotel_df
hotel_reset = hotel_reset.reset_index()
hotel_reset

,index,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,248,Dezful,32.3811,48.4058,74.39,20,0,4.88,IR,12600
1,293,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,14400
2,326,Atar,20.5169,-13.0499,79.54,17,0,7.09,MR,0
3,382,Camargo,27.6667,-105.1667,79.95,18,0,3.53,MX,-21600


In [16]:
for x in range (0,len(hotel_reset['City Name'])):
    hotel_reset.loc[[x],["Hotel Name"]] = hotel_list[x]
    hotel_reset.loc[[x],["Hotel Lat"]] = hlat[x]
    hotel_reset.loc[[x],["Hotel Lng"]] = hlng[x]
hotel_reset=hotel_reset.rename(columns={'Latitude':'Lat','Longitude':'Lng',"City Name":"City"})
hotel_reset

,index,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name,Hotel Lat,Hotel Lng
0,248,Dezful,32.3811,48.4058,74.39,20,0,4.88,IR,12600,Dezful Tourism Hotel,32.390944,48.383530
1,293,Salalah,17.0151,54.0924,73.49,64,0,5.75,OM,14400,Salalah Gardens Hotel,17.022848,54.064693
2,326,Atar,20.5169,-13.0499,79.54,17,0,7.09,MR,0,Auberge et Camping Inimi,20.501348,-13.060137
3,382,Camargo,27.6667,-105.1667,79.95,18,0,3.53,MX,-21600,Hotel Santa Fe,27.670087,-105.156801


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_reset.iterrows()]
locations = hotel_reset[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…